<a href="https://colab.research.google.com/github/dpcks/playdata/blob/main/DeepLearning/13_%EB%8B%A4%EC%A4%91%EB%B6%84%EB%A5%98%EC%97%B0%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이터 경로를 변경하시오

In [ ]:
DATA_PATH = "/content/drive/MyDrive/딥러닝/data/"
SEED = 42

In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random # 시드 고정을 위해
import os # 시드 고정을 위해
from PIL import Image

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)    # 파이썬 환경변수 시드 고정
    np.random.seed(seed)
    torch.manual_seed(seed) # cpu 연산 무작위 고정
    torch.cuda.manual_seed(seed) # gpu 연산 무작위 고정
    torch.backends.cudnn.deterministic = True  # cuda 라이브러리에서 Deterministic(결정론적)으로 예측하기 (예측에 대한 불확실성 제거 )


device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 데이터 복사및 압축풀기

In [ ]:
if not os.path.isdir("train"):
    !cp "{DATA_PATH}meat.zip" "meat.zip"
    !unzip -qq "meat.zip" 

# 이미지경로 및 정답 데이터셋

In [ ]:
train = pd.read_csv("train/class_info.csv")
test = pd.read_csv("test/class_info.csv")
train.shape , test.shape

((1246, 2), (1020, 2))

# 정답값
- 0 : 신선한고기
- 1 : 반 신선한 고기
- 2 : 상태 안좋은 고기

In [ ]:
train

,filename,target
0,1074.jpg,0
1,1222.jpg,1
2,2105.jpg,1
3,81.jpg,2
4,1791.jpg,2
...,...,...
1241,1670.jpg,1
1242,1244.jpg,0
1243,776.jpg,2
1244,750.jpg,0


# transforms 객체

In [ ]:
from torchvision import transforms

img_size = [224,224]

train_lst = [
    transforms.Resize(img_size),
    transforms.RandomHorizontalFlip(p=0.4),# 증강함수 사용해주기
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
train_transform = transforms.Compose(train_lst)

test_lst = [
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]
test_transform = transforms.Compose(test_lst)

# 데이터셋

- 다중분류에서는 1차원텐서로 넣어줘야함

In [ ]:
class MeatDataset(torch.utils.data.Dataset):
    def __init__(self ,transform , x , is_inference = False ): 
        self.is_inference = is_inference
        self.transform = transform
        if not self.is_inference: # false이면 학습모드
            self.y = x["target"].to_numpy() # 다중분류에서는 정답값을 내부적으로 원핫 인코딩하기 때문에 1차원형태로 전달해줘야하고, int64 형태로 전달해줘야한다.
            self.x = "train/" + x["filename"]
        else:
            self.x = "test/" + x["filename"]
    def __len__(self): 
        return self.x.shape[0]
    def __getitem__(self, idx): 
        item = {}
        x = Image.open(self.x[idx])
        item["x"] = self.transform(x)
        if not self.is_inference:
            item["y"] = torch.tensor(self.y[idx]) # 넘파이 데이터 타입 그대로 유지하기위해 tensor 함수를 사용한다.
        return item   

In [ ]:
dt = MeatDataset(train_transform,train)
dl = torch.utils.data.DataLoader(dt, batch_size=2, shuffle=True)
batch  = next(iter(dl))
batch

In [ ]:
from torchvision.models import resnet50 , ResNet50_Weights 
from torchvision.models import swin_t , Swin_T_Weights 

In [ ]:
swin_t(weights = Swin_T_Weights.IMAGENET1K_V1)

SwinTransformer(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): Permute()
      (2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
    )
    (1): Sequential(
      (0): SwinTransformerBlock(
        (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (attn): ShiftedWindowAttention(
          (qkv): Linear(in_features=96, out_features=288, bias=True)
          (proj): Linear(in_features=96, out_features=96, bias=True)
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
        (norm2): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (0): Linear(in_features=96, out_features=384, bias=True)
          (1): GELU(approximate=none)
          (2): Dropout(p=0.0, inplace=False)
          (3): Linear(in_features=384, out_features=96, bias=True)
          (4): Dropout(p=0.0, inplace=False)
        )
      )
      (1): SwinTransformerBlock(
        (

In [ ]:
class Net(torch.nn.Module):
    def __init__(self): 
        super().__init__()
        self.pre_model1 = swin_t(weights = Swin_T_Weights.IMAGENET1K_V1)
        self.pre_model2 = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        # self.pre_model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V2)
        self.pre_model1.head = torch.nn.Linear(768,512) # 예측을 3개 맞춰야하기때문
        self.pre_model2.fc = torch.nn.Linear(2048,1024)
        self.dropout = torch.nn.Dropout(0.2)
        self.output_layer = torch.nn.Linear(1536,3)

    def forward(self, x):
        x1 = self.pre_model1(x)
        x2 = self.pre_model2(x)
        x  = torch.cat((x1,x2),dim=1)
        x  = self.dropout(x)
        x  = self.output_layer(x)
        return x

In [ ]:
model = Net()
model(batch["x"])

tensor([[-0.0340,  0.0286, -0.0589],
        [ 0.2502,  0.0178, -0.2234]], grad_fn=<AddmmBackward0>)

In [ ]:
!pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import torchinfo
model = Net()
torchinfo.summary(model,(32,3,224,224))  

Layer (type:depth-idx)                                  Output Shape              Param #
Net                                                     [32, 3]                   --
├─SwinTransformer: 1-1                                  [32, 512]                 --
│    └─Sequential: 2-1                                  [32, 7, 7, 768]           --
│    │    └─Sequential: 3-1                             [32, 56, 56, 96]          4,896
│    │    └─Sequential: 3-2                             [32, 56, 56, 96]          224,694
│    │    └─PatchMerging: 3-3                           [32, 28, 28, 192]         74,496
│    │    └─Sequential: 3-4                             [32, 28, 28, 192]         891,756
│    │    └─PatchMerging: 3-5                           [32, 14, 14, 384]         296,448
│    │    └─Sequential: 3-6                             [32, 14, 14, 384]         10,658,952
│    │    └─PatchMerging: 3-7                           [32, 7, 7, 768]           1,182,720
│    │    └─Sequential:

In [ ]:
def train_loop(dataloader,model,loss_fn,optimizer,device):
    epoch_loss = 0 
    model.train() 
    for batch in dataloader: 
        pred = model(batch["x"].to(device)) 
        loss = loss_fn(pred, batch["y"].to(device)) 
        
        optimizer.zero_grad() 
        loss.backward()  
        optimizer.step() 
        
        epoch_loss += loss.item() 

    epoch_loss /= len(dataloader) 

    return epoch_loss 

In [ ]:
@torch.no_grad() 
def test_loop(dataloader,model,loss_fn,device): 
    epoch_loss = 0
    model.eval() 

    
    pred_list = []
    softmax = torch.nn.Softmax(dim=1) # 다중분류 예측 확률을 출력하기위해 softmax 사용, 회귀문제였다면 없어도댐

    for batch in dataloader:
        
        pred = model(batch["x"].to(device))
        if batch.get("y") is not None: 
            loss = loss_fn(pred, batch["y"].to(device))
            epoch_loss += loss.item()
        
        pred = softmax(pred)
        pred = pred.to("cpu").numpy() 
        pred_list.append(pred)

    epoch_loss /= len(dataloader)

    pred = np.concatenate(pred_list) 
    return epoch_loss , pred 

# 일부 동결
- 고기는 일부동결하니까 더 떨어짐 0.984


In [ ]:
# model = Net()
# model

In [ ]:
# for name, param in model.named_parameters():
#     print(name,param.requires_grad)

In [ ]:
# model = Net()
# for name, param in model.named_parameters():
#     if name.startswith("pre_model.layer3"):
#         break
#     param.requires_grad = False

In [ ]:
n_splits = 5
batch_size = 32 
epochs = 100
loss_fn = torch.nn.CrossEntropyLoss() # 다중분류 손실객체

In [ ]:
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold
cv = KFold(n_splits=n_splits,shuffle=True, random_state=SEED)

In [ ]:
is_holdout = True
reset_seeds(SEED)
best_score_list = []
for i,(tri,vai) in enumerate(cv.split(train)):
    
    model = Net().to(device)
    optimizer = torch.optim.Adam(model.parameters(),lr=0.0001)
    
    x_train = train.iloc[tri].reset_index(drop=True) # 학습 데이터 프레임, 인덱스가 흐트러지기떄문에 reset_index 인덱스 정렬하는 습관을 들여라!
    x_valid = train.iloc[vai].reset_index(drop=True) # 검증 데이터 프레임

    train_dt = MeatDataset(train_transform,x_train)
    valid_dt = MeatDataset(test_transform,x_valid)
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size,shuffle=False)

    best_score = 0
    patience = 0

    for epoch in tqdm(range(epochs)):
        
        train_loss = train_loop(train_dl, model, loss_fn,optimizer,device )
        valid_loss , pred = test_loop(valid_dl, model, loss_fn,device  )
        
        pred = np.argmax(pred, axis=1) # 가장 확률이 높은 인덱스를 정답값으로 결정
        true = valid_dt.y # y 인스턴스 변수에 정답값이 있음
        score = f1_score(true, pred , average="micro")
        print(train_loss,valid_loss,score)
        patience += 1
        if best_score < score:
            patience = 0
            best_score = score
            torch.save(model.state_dict(),f"model_{i}.pth")

        if patience == 5:
            break
    print(f"Fold ({i}), BEST F1: {best_score}")
    best_score_list.append(best_score)

    if is_holdout:
        break

  0%|          | 0/100 [00:00<?, ?it/s]

0.4097107562702149 0.1481275800615549 0.944
0.05522358624875778 0.019908253278117627 0.996
0.029401572362985462 0.010952422824630048 0.996
0.01101626367380959 0.013627399068354862 0.996
0.010058776024379767 0.006058800900063943 0.996
0.019347587800439214 0.011150261914735893 0.992
0.006097366603171395 0.013590980186563684 0.996
Fold (0), BEST F1: 0.996
